In [1]:
import tensorflow as tf
import scipy.io as scp
import numpy as np
import random

In [2]:
testData = scp.loadmat('../data/svhn/test_32x32.mat')
trainData = scp.loadmat('../data/svhn/train_32x32.mat')

In [3]:
testDataX = testData['X']
testDataY = testData['y']

trainDataX = trainData['X']
trainDataY = trainData['y']

In [4]:
print type(trainDataX)
print type(trainDataY)

print trainDataX.shape
print trainDataY.shape
print testDataX.shape
print testDataY.shape

<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
(32, 32, 3, 73257)
(73257, 1)
(32, 32, 3, 26032)
(26032, 1)


In [5]:
# try tansposing the array
def transposeArray(data):
    print 'started'
    xtrain = []
    trainLen = data.shape[3]
    print trainLen
    for x in xrange(trainLen):
        xtrain.append(data[:,:,:,x])
    
    xtrain = np.asarray(xtrain)
    return xtrain

In [6]:
trainDataX = transposeArray(trainDataX)
testDataX = transposeArray(testDataX)


print trainDataX.shape

started
73257
started
26032
(73257, 32, 32, 3)


In [7]:
def OnehotEndoding(Y):
    Ytr=[]
    for el in Y:
        temp=np.zeros(10)
        if el==10:
            temp[0]=1
        elif el==1:
            temp[1]=1
        elif el==2:
            temp[2]=1
        elif el==3:
            temp[3]=1
        elif el==4:
            temp[4]=1
        elif temp[5]==1:
            temp[5]=1
        elif temp[6]==1:
            temp[6]=1
        elif temp[7]==1:
            temp[7]=1
        elif temp[8]==1:
            temp[8]=1
        elif temp[9]==1:
            temp[9]=1
        Ytr.append(temp)
        
    return np.asarray(Ytr)

In [8]:
# convert y to one hot encoding
trainDataY = OnehotEndoding(trainDataY)
testDataY = OnehotEndoding(testDataY)
print trainDataY.shape
print testDataY.shape

(73257, 10)
(26032, 10)


In [9]:
height = 32
width = 32
channel = 3
tags = 10
patch = 5
depth = 16
num_hidden = 64

learning_rate = 0.00001

In [10]:
tf_X = tf.placeholder("float", shape=[None, height, width, channel])
tf_Y = tf.placeholder("float", shape=[None, tags])

convW1 = tf.Variable(tf.random_normal([patch, patch, channel, depth], stddev=0.1))
bias1 = tf.Variable(tf.random_normal([depth], stddev=0.1))

convW2 = tf.Variable(tf.random_normal([patch, patch, depth, depth], stddev=0.1))
bias2 = tf.Variable(tf.random_normal([depth], stddev=0.1))

w3 = tf.Variable(tf.random_normal([height // 4 * width // 4 * depth, num_hidden], stddev=0.1))
bias3 = tf.Variable(tf.random_normal([num_hidden]))

w4 = tf.Variable(tf.random_normal([num_hidden, tags], stddev=0.1))
bias4 = tf.Variable(tf.random_normal([tags], stddev=0.1))              


In [11]:
#model
def model(X):
    conv = tf.nn.conv2d(X, convW1, [1,2,2,1], padding='SAME')
    hidden1 = tf.nn.relu(conv + bias1)
    
    conv2 = tf.nn.conv2d(hidden1, convW2, [1,2,2,1], padding='SAME')
    hidden2 = tf.nn.relu(conv2 + bias2)
    
    #reshape it to a single Dimensional
    shape = hidden2.get_shape()
    print hidden2.get_shape().as_list()
    
    newInput = tf.reshape(hidden2, [-1, shape[1].value * shape[2].value * shape[3].value])
    hidden3 = tf.nn.relu(tf.matmul(newInput, w3) + bias3)
    
    return tf.matmul(hidden3, w4) + bias4

In [12]:
pred = model(tf_X)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, tf_Y))

# Optimizer.
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(pred,1),tf.argmax(tf_Y,1)), "float"))

[None, 8, 8, 16]


In [13]:
def Accuracy(X, Y, message, sess):    
    print message, sess.run(accuracy, feed_dict= {tf_X: X, tf_Y: Y})

In [15]:
with tf.Session() as sess:
    tf.initialize_all_variables().run()
    epoch = 10000
    batch_size = 100
    print('Initialized')
    
    p = np.random.permutation(range(len(trainDataX)))
    trX, trY = trainDataX[p], trainDataY[p]
    
    for step in range(epoch):
        
        batch = np.random.choice(len(trainDataX) - 1, batch_size)
        inX, outY = trainDataX[batch], trainDataY[batch]
        sess.run(optimizer, feed_dict= {tf_X: inX, tf_Y: outY})
        
        if step % 50 == 0:
            print 'cost at each step :', step, 'is :', sess.run(loss, feed_dict={tf_X: inX, tf_Y: outY})
    
    Accuracy(trainDataX, trainDataY, 'accuracy of training data : ', sess)
    Accuracy(testDataX, testDataY, 'accuracy of test data : ', sess)

 Initialized
cost at each step : 0 is : 245.22
cost at each step : 50 is : 19.9122
cost at each step : 100 is : 19.2203
cost at each step : 150 is : 22.6261
cost at each step : 200 is : 101.576
cost at each step : 250 is : 6278.88
cost at each step : 300 is : 29.0328
cost at each step : 350 is : 12.0666
cost at each step : 400 is : 5.65341
cost at each step : 450 is : 3.81649
cost at each step : 500 is : 3.66395
cost at each step : 550 is : 2.66991
cost at each step : 600 is : 3.32461
cost at each step : 650 is : 2.14366
cost at each step : 700 is : 1.96371
cost at each step : 750 is : 1.76727
cost at each step : 800 is : 2.39551
cost at each step : 850 is : 1.67991
cost at each step : 900 is : 2.11456
cost at each step : 950 is : 1.72611
cost at each step : 1000 is : 1.99279
cost at each step : 1050 is : 1.92433
cost at each step : 1100 is : 1.52921
cost at each step : 1150 is : 1.6069
cost at each step : 1200 is : 1.69033
cost at each step : 1250 is : 1.68714
cost at each step : 1300